# Consumer

## Read data with kafka consumer

In [2]:
from confluent_kafka import Consumer

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Configuração do consumidor
kafka_broker = "kafka-cpc.certi.org.br:31289"
topic_name = "iot-temperature"

consumer_config = {
    'bootstrap.servers': kafka_broker,
    'group.id': 'my-iot-group',
    'auto.offset.reset': 'earliest'
}
consumer = Consumer(consumer_config)

consumer.subscribe([topic_name])

print("Monitorando mensagens do Kafka... Pressione Ctrl+C para parar.")

try:
    while True:
        msg = consumer.poll(1.0)  # Aguarda por mensagens

        if msg is None:
            continue
        if msg.error():
            print(f"Erro: {msg.error()}")
            continue

        print(f"Recebido: {msg.value().decode('utf-8')} de {msg.topic()} [{msg.partition()}]")

except KeyboardInterrupt:
    print("Parando o consumidor...")
finally:
    consumer.close()


Monitorando mensagens do Kafka... Pressione Ctrl+C para parar.
Parando o consumidor...


## Read data with spark structured streaming

Create Delta Lake table with defined schema and change data feed enabled

In [1]:
from pyspark.sql.types import StructType, StringType, DoubleType
from pyspark.sql.functions import from_json, col
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, DoubleType
import os

os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages org.apache.hadoop:hadoop-aws:3.3.4,io.delta:delta-spark_2.12:3.3.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.4,org.apache.kafka:kafka-clients:3.9.0 pyspark-shell"

# Initialize Spark session with Delta Lake and MinIO support
spark = (SparkSession.builder \
    .appName("DeltaLakeWithMinIO") \
    ## Delta
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    #Hive Catalog
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    ## Optimize Delta
    .config("delta.autoOptimize.optimizeWrite", "true") \
    .config("delta.autoOptimize.autoCompact", "true") \
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
    ## MinIO
    .config("spark.hadoop.fs.s3a.endpoint", os.getenv("MINIO_ENDPOINT")) \
    .config("spark.hadoop.fs.s3a.access.key", os.getenv("MINIO_ACCESS_KEY")) \
    .config("spark.hadoop.fs.s3a.secret.key", os.getenv("MINIO_SECRET_KEY")) \
    .config('spark.hadoop.fs.s3a.attempts.maximum', "3") \
    .config('spark.hadoop.fs.s3a.connection.timeout', "10000") \
    .config('spark.hadoop.fs.s3a.connection.establish.timeout', "5000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3n.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate())

your 131072x1 screen size is bogus. expect trouble
25/01/16 13:34:03 WARN Utils: Your hostname, CPC-12806 resolves to a loopback address: 127.0.1.1; using 172.26.242.248 instead (on interface eth0)
25/01/16 13:34:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/mnt/c/Users/ark/Meu%20Drive/Projetos%20Internos/spark-transform/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/arthur/.ivy2/cache
The jars for the packages stored in: /home/arthur/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-spark_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-532d8831-9d5b-48e0-9e74-97305d45db2d;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.4 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.

In [2]:
from pyspark.sql.types import StructType, StringType, DoubleType

# Define the schema for the processed table
processed_schema = StructType() \
    .add("timestamp", StringType()) \
    .add("temperature", DoubleType())

# Create an empty DataFrame with the schema
empty_processed_df = spark.createDataFrame([], processed_schema)

# Write the empty DataFrame to create the Delta table with CDF enabled
empty_processed_df.write.format("delta") \
    .option("path", "s3a://lakehouse/delta/raw_iot_data") \
    .option("delta.enableChangeDataFeed", "true") \
    .mode("overwrite") \
    .save()

25/01/16 13:34:20 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [3]:
spark.sql("SELECT COUNT(*) FROM delta.`s3a://lakehouse/delta/raw_iot_data`;").show()

25/01/16 13:34:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+
|count(1)|
+--------+
|       0|
+--------+



## Read Kafka Stream

In [6]:
# Kafka Configuration
kafka_broker = "kafka-cpc.certi.org.br:31289"
topic_name = "iot-temperature"

# Define the schema for the JSON data
schema = StructType() \
    .add("timestamp", StringType()) \
    .add("temperature", DoubleType())

# Read data from Kafka
kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_broker) \
    .option("subscribe", topic_name) \
    .option("startingOffsets", "earliest") \
    .load()

# Deserialize Kafka value (JSON string) into columns
parsed_stream = kafka_stream.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")

# Output the parsed stream for verification
parsed_stream.printSchema()


root
 |-- timestamp: string (nullable = true)
 |-- temperature: double (nullable = true)



## Write Delta Stream

In [7]:
# Define the path to the raw Delta table
raw_delta_path = "s3a://lakehouse/delta/raw_iot_data"

# Write Kafka stream to Delta table
raw_stream_query = parsed_stream.writeStream \
    .format("delta") \
    .option("checkpointLocation", "s3a://lakehouse/delta/checkpoints/raw_iot_data") \
    .outputMode("append") \
    .start(raw_delta_path)

print("Streaming Kafka data into Delta Lake (raw_iot_data)...")

# Monitor the query progress
import time

while raw_stream_query.isActive:
    print(raw_stream_query.lastProgress)  # Shows the latest progress info
    time.sleep(5)  # Updates every 5 seconds


25/01/16 14:00:11 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/01/16 14:00:12 WARN StreamingQueryManager: Stopping existing streaming query [id=c4c5e95a-a077-482c-8787-76257b98db8f, runId=ee839e49-6f9f-49d0-94ff-6c678438328a], as a new run is being started.


Streaming Kafka data into Delta Lake (raw_iot_data)...
None


None


{'id': 'c4c5e95a-a077-482c-8787-76257b98db8f', 'runId': '04edde25-e713-41ae-852c-bde3ae79f0ce', 'name': None, 'timestamp': '2025-01-16T17:00:12.426Z', 'batchId': 126, 'numInputRows': 9, 'inputRowsPerSecond': 0.0, 'processedRowsPerSecond': 1.025524156791249, 'durationMs': {'addBatch': 4811, 'commitOffsets': 1331, 'getBatch': 0, 'latestOffset': 387, 'queryPlanning': 119, 'triggerExecution': 8776, 'walCommit': 1481}, 'stateOperators': [], 'sources': [{'description': 'KafkaV2[Subscribe[iot-temperature]]', 'startOffset': {'iot-temperature': {'8': 348, '11': 359, '2': 376, '5': 398, '14': 394, '13': 352, '4': 370, '7': 355, '1': 356, '10': 369, '9': 360, '3': 379, '12': 406, '6': 368, '0': 356}}, 'endOffset': {'iot-temperature': {'8': 348, '11': 360, '2': 376, '5': 399, '14': 394, '13': 352, '4': 371, '7': 356, '1': 356, '10': 369, '9': 360, '3': 380, '12': 406, '6': 371, '0': 357}}, 'latestOffset': {'iot-temperature': {'8': 348, '11': 360, '2': 376, '5': 399, '14': 394, '13': 352, '4': 371,

{'id': 'c4c5e95a-a077-482c-8787-76257b98db8f', 'runId': '04edde25-e713-41ae-852c-bde3ae79f0ce', 'name': None, 'timestamp': '2025-01-16T17:00:12.426Z', 'batchId': 126, 'numInputRows': 9, 'inputRowsPerSecond': 0.0, 'processedRowsPerSecond': 1.025524156791249, 'durationMs': {'addBatch': 4811, 'commitOffsets': 1331, 'getBatch': 0, 'latestOffset': 387, 'queryPlanning': 119, 'triggerExecution': 8776, 'walCommit': 1481}, 'stateOperators': [], 'sources': [{'description': 'KafkaV2[Subscribe[iot-temperature]]', 'startOffset': {'iot-temperature': {'8': 348, '11': 359, '2': 376, '5': 398, '14': 394, '13': 352, '4': 370, '7': 355, '1': 356, '10': 369, '9': 360, '3': 379, '12': 406, '6': 368, '0': 356}}, 'endOffset': {'iot-temperature': {'8': 348, '11': 360, '2': 376, '5': 399, '14': 394, '13': 352, '4': 371, '7': 356, '1': 356, '10': 369, '9': 360, '3': 380, '12': 406, '6': 371, '0': 357}}, 'latestOffset': {'iot-temperature': {'8': 348, '11': 360, '2': 376, '5': 399, '14': 394, '13': 352, '4': 371,

...

{'id': 'c4c5e95a-a077-482c-8787-76257b98db8f', 'runId': '04edde25-e713-41ae-852c-bde3ae79f0ce', 'name': None, 'timestamp': '2025-01-16T17:04:01.977Z', 'batchId': 158, 'numInputRows': 8, 'inputRowsPerSecond': 1.0562450488513335, 'processedRowsPerSecond': 1.1885306789481505, 'durationMs': {'addBatch': 4178, 'commitOffsets': 1287, 'getBatch': 0, 'latestOffset': 111, 'queryPlanning': 5, 'triggerExecution': 6731, 'walCommit': 1150}, 'stateOperators': [], 'sources': [{'description': 'KafkaV2[Subscribe[iot-temperature]]', 'startOffset': {'iot-temperature': {'8': 366, '11': 372, '2': 389, '5': 417, '14': 406, '13': 365, '4': 386, '7': 371, '1': 366, '10': 380, '9': 373, '3': 395, '12': 429, '6': 388, '0': 373}}, 'endOffset': {'iot-temperature': {'8': 368, '11': 372, '2': 390, '5': 418, '14': 406, '13': 366, '4': 386, '7': 371, '1': 366, '10': 380, '9': 374, '3': 396, '12': 429, '6': 389, '0': 373}}, 'latestOffset': {'iot-temperature': {'8': 368, '11': 372, '2': 390, '5': 418, '14': 406, '13': 

{'id': 'c4c5e95a-a077-482c-8787-76257b98db8f', 'runId': '04edde25-e713-41ae-852c-bde3ae79f0ce', 'name': None, 'timestamp': '2025-01-16T17:04:08.708Z', 'batchId': 159, 'numInputRows': 7, 'inputRowsPerSecond': 1.0399643440796316, 'processedRowsPerSecond': 0.7244127082686536, 'durationMs': {'addBatch': 6960, 'commitOffsets': 1291, 'getBatch': 0, 'latestOffset': 123, 'queryPlanning': 16, 'triggerExecution': 9663, 'walCommit': 1273}, 'stateOperators': [], 'sources': [{'description': 'KafkaV2[Subscribe[iot-temperature]]', 'startOffset': {'iot-temperature': {'8': 368, '11': 372, '2': 390, '5': 418, '14': 406, '13': 366, '4': 386, '7': 371, '1': 366, '10': 380, '9': 374, '3': 396, '12': 429, '6': 389, '0': 373}}, 'endOffset': {'iot-temperature': {'8': 368, '11': 372, '2': 391, '5': 418, '14': 406, '13': 366, '4': 386, '7': 371, '1': 367, '10': 381, '9': 377, '3': 396, '12': 429, '6': 389, '0': 374}}, 'latestOffset': {'iot-temperature': {'8': 368, '11': 372, '2': 391, '5': 418, '14': 406, '13':

{'id': 'c4c5e95a-a077-482c-8787-76257b98db8f', 'runId': '04edde25-e713-41ae-852c-bde3ae79f0ce', 'name': None, 'timestamp': '2025-01-16T17:04:18.372Z', 'batchId': 160, 'numInputRows': 9, 'inputRowsPerSecond': 0.9312913907284769, 'processedRowsPerSecond': 1.300578034682081, 'durationMs': {'addBatch': 4373, 'commitOffsets': 1173, 'getBatch': 0, 'latestOffset': 125, 'queryPlanning': 4, 'triggerExecution': 6920, 'walCommit': 1245}, 'stateOperators': [], 'sources': [{'description': 'KafkaV2[Subscribe[iot-temperature]]', 'startOffset': {'iot-temperature': {'8': 368, '11': 372, '2': 391, '5': 418, '14': 406, '13': 366, '4': 386, '7': 371, '1': 367, '10': 381, '9': 377, '3': 396, '12': 429, '6': 389, '0': 374}}, 'endOffset': {'iot-temperature': {'8': 368, '11': 372, '2': 392, '5': 419, '14': 406, '13': 366, '4': 386, '7': 371, '1': 367, '10': 383, '9': 379, '3': 397, '12': 431, '6': 389, '0': 374}}, 'latestOffset': {'iot-temperature': {'8': 368, '11': 372, '2': 392, '5': 419, '14': 406, '13': 3

25/01/16 14:04:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


KeyboardInterrupt: 